# Assignment 4: Explainability

*Part of the course:
Machine Learning (code: INFOB3ML), Fall 2025, Utrecht University*

Total points: 10 (100%)

Deadline: Friday 31 October, 23:59

The stundets should submit one ipynb file per pair.

**Write your names and student numbers here: ___**

Submit one ipynb file per pair.

**Before you submit, click Kernel > Restart & Run All to make sure you submit a working version of your code!**



## Installation

For this assignment, we are going to use the following Python packages:

matplotlib, pandas, statsmodels, interpret, scikit-learn, openpyxl and graphviz

In [1]:
# Installing packages
%pip install graphviz plotnine seaborn
%pip install matplotlib pandas numpy statsmodels scikit-learn openpyxl
%pip install interpret
%pip install scikit-learn --upgrade

  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached contourpy-1.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-12.0.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (8.7 MB)
Using ca

## Downloading the data
We are going to use the combined cycle power plant dataset. This dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. We have the following features: hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V). We will train ML models to predict the net hourly electrical energy output (EP) of the plant.

For a detailed description, see: [[Description](https://archive.ics.uci.edu/dataset/294/combined+cycle+power+plant)]

We first need to download and prepare data.


In [1]:
# Download and unzip data

# Windows users: download the data from https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
# and unzip the file manually in the same folder as the python notebook

# Note: this cell works fine on Linux based systems and Google Colab
# If you run it on a Windows machine, you will get an error (...'wget' is not recognized as an internal or external command...)
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
!unzip CCPP.zip

--2025-10-28 12:38:11--  https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘CCPP.zip’

CCPP.zip                [               <=>  ]   3.50M  72.7KB/s    in 46s     

2025-10-28 12:38:57 (78.5 KB/s) - ‘CCPP.zip’ saved [3674852]

Archive:  CCPP.zip
   creating: CCPP/
  inflating: CCPP/Folds5x2_pp.ods    
  inflating: CCPP/Folds5x2_pp.xlsx   
  inflating: CCPP/Readme.txt         
  inflating: CCPP/Readme.txt~        


## Loading and preprocessing the data
We split the data into training (first 5000 instances) and validation (the subsequent 2000) and test (the last 2568) sets. We will use the training set to train a model, and validation set to optimize the model hyper-parameters.


In [3]:
# Load and prepare data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# global variables
DATA_FILENAME = 'CCPP/Folds5x2_pp.xlsx'
FEATURE_NAMES = ['AT', 'V', 'AP', 'RH']
LABEL_NAME = 'PE'
# Load the data from the excel file
def load_data():
    def split_feature_label(data_set):
        features = data_set[FEATURE_NAMES]
        labels = data_set[LABEL_NAME]
        return features, labels

    data = pd.read_excel(DATA_FILENAME)
    train_set, dev_set, test_set = data[:5000], data[5000: 7000], data[7000:]

    train_features, train_labels = split_feature_label(train_set)
    dev_features, dev_labels = split_feature_label(dev_set)
    test_features, test_labels = split_feature_label(test_set)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels


# preprocess (by z-normalization) the data for the regression task
# return the normalized feature sets and corresponding target variables
def prepare_load_regression_data():
    train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels = load_data()

    scaler = StandardScaler()
    scaler = scaler.fit(train_features)
    train_features = pd.DataFrame(data=scaler.transform(train_features), columns=FEATURE_NAMES)
    dev_features = pd.DataFrame(data=scaler.transform(dev_features), columns=FEATURE_NAMES)
    test_features = pd.DataFrame(data=scaler.transform(test_features), columns=FEATURE_NAMES)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels



## Training and Interpreting a Linear Regression Model

**Q1**. (10)% Train a linear regression model (we recommend the statsmodels package) and report $R^2$ (goodness of fit) statistic.

For model interpretability, provide for each feature (+ the bias variable) the following in tabular format:
* Weight estimates
* SE (standard error of estimates)
* T statistics


Further Questions regarding the linear model (answers to be included in the notebook):

**Q2**. (5%) Which three features are the most important?

**Q3**. (10%) How does the gas turbine energy yield (EP) change with unit (one degree C) increase of the ambient temperature given that all other feature values remain the same? (Note: Here you should consider whether you use the original or z-normalized features to train your linear model.)

**Q4**. (10%) Show bar graph illustrations of the feature effects for the first two validation set instances.

In [4]:
# We recommend the statsmodels package
import statsmodels.api as sm
# Hint, by default this sm does not include the bias/offset term w_0
# thus, you should add it yourself using sm.add_constant()

# Linear regression
# Get the preprocessed data for linear regression modeling
(train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels) = prepare_load_regression_data()


**Q5**. (5%) Reflection: why would training a regression tree not work well for this dataset in terms of model interpretability? (answer in the notebook)

## Training and Interpreting an Explainable Boosting Model (Generalized Additive Model)
**Q6**. (20%) (20%) Train a Explainable Boosting Machine (with [interpret.ml](https://interpret.ml/docs/ebm.html))

(Note on grading: Training EBM 5%, answering the questions - see below -  5% each)

For a documentation see: [[LIME](https://interpret.ml/docs/lime.html)]

* (5%) Visualize/provide global (model-wise) feature importances for EBM as a table or figure.
* (5%) What are the most important two features in EBM? Are they the same as in the linear model?
* (5%) Visualize local (instance-wise) feature importances for a development set instance of your choice.


In [ ]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from interpret.glassbox import ExplainableBoostingRegressor
# add any other imports you need below


# EBM

## Training and Explaining Neural Networks
**Q7**. (15%) Train a Neural Network (using the training and validation sets): One-layer MLP (ReLU activation function + 50 hidden neurons)

We recommend to use the Adam optimizer. Fine-tune the learning rate and any other hyper-parameters you find necessary.

For a tutorial see: [[Tutorial](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)]

Your code should report the results following the instructions below:

Note on grading: training NN: 5%, answering below sub-questions 5% each.

* (5%) Apply the trained neural network model on the test set and report Root Mean Square Error (RMSE) performance measure.

* (5%) Analyzing factors influencing the neural network predictions.
See the [Documentation](https://scikit-learn.org/stable/modules/partial_dependence.html) to use Partial Dependence Plot (PDP)  implementation in python. Use the trained one-layer MLP model to  Generate and report a bivariate PDP using 'AT' (Ambient Temperature) and 'V' (Exhaust Vacuum) features (Note: not two univariate PDPs but one bivariate PDP).


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.inspection import PartialDependenceDisplay

# One-layer MLP : you can use  learning_rate_init=0.001 with relu activation to get a reasonable model,
# optimize other parameters by experimentation
# We advice that you name variable for the mlp regressor model 'mlp_reg' so that it will be consistent
# with the scripts to call your implementation of PFI later in Q8:
mlp_reg= # ... your scripts here and below

### Generating Model-Agnostic Global Explanations for NN predictions

**Permutation Feature Importance (PFI)**

**Q8**. (25%) PFI implementation and testing
- (%15) Implement the permutation feature importance algorithm using RMSE as the error function. No existing libraries (save measuring RMSE) are allowed to use. We will implement it ourselves. Hint: to get a better error estimate per feature perturbation, you can run feature permutation multiple times (e.g., 10 times) and calculate the average.
- (%5) Visualize feature importances obtained by PFI for the NN (one-layer MLP) model you trained using a bar graph.
- (%5) Reflection: What are the most important two features obtained by PFI for MLP model? How do these two features compare to the top two features from the Linear Model and the EBM?

In [ ]:
# Use the below function signature and the parameters to implement PFI
def PFI(X, labels, model, orig_rmse):
# Your scripts implementing PFI here




# Do not modify this part (unless you change the variable name mlp_reg above)
base_rmse = root_mean_squared_error(dev_labels, mlp_reg.predict(dev_features))
results_df = PFI(dev_features, dev_labels, mlp_reg, base_rmse)

In [ ]:
# Your scripts to visualize results_df using a bar graph here